In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [ ]:
df_test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
df_train.head(10)

In [ ]:
df_train['target'].value_counts()

In [ ]:
df_train['location'].value_counts()

In [ ]:
df_train['keyword'].value_counts()

In [ ]:
import seaborn as sns

# df_train.corr()

In [ ]:
list(df_train['text'].head(10))

In [ ]:
pattern = r'http://\S*'
pattern1 = r'https://\S*'

import re
# Remove links from strings
filtered_strings = [re.sub(pattern, '', re.sub(pattern1, '', s.lower())) for s in list(df_train['text'])]

In [ ]:
filtered_strings[0]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in filtered_strings:
	val = str(val)
	# split the value
	tokens = val.split()
	
	# Converts each token into lowercase
	for i in range(len(tokens)):
		tokens[i] = tokens[i].lower()
	
	comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800,
				background_color ='white',
				stopwords = stopwords,
				min_font_size = 10).generate(comment_words)

# plot the WordCloud image					
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()


In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
stop_words=set(nltk.corpus.stopwords.words('english'))
print(stop_words)

In [ ]:
filtered_strings=[re.sub("[^a-zA-Z]"," ",f) for f in filtered_strings]
# word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]
words_string=[]
for f in filtered_strings:
    words=f.split(" ")
    word_final=""
    for w in words:
        if w not in stopwords:
            word_final+=w+" "
    word_final=word_final[:-1]
    words_string.append(word_final)
filtered_strings=words_string

In [ ]:
filtered_strings[0]

In [ ]:
tokenizer = TweetTokenizer()
tokens=[]
for ft in filtered_strings:
#         review_text = re.sub("[^a-zA-Z]"," ",review_text)

    tokens.append(tokenizer.tokenize(ft)) 
# tokens=list(set(tokens))
# print(tokens)

# vocab = {token: i for i, token in enumerate(set(tokens))}
tokens[0:2]


In [ ]:
maxLen=max([len(i) for i in tokens])
maxLen

In [ ]:
#Word Embeddings Word2Vec

from gensim.models import Word2Vec

# sentences = [["I", "love", "machine", "learning"], ["This", "is", "a", "sentence"]]
wv_model = Word2Vec(tokens, vector_size=100, window=5, min_count=1, workers=4)
# wv_model.train(tokens,epochs=10,total_examples=len(tokens))


In [ ]:
wv_model.wv['alert'].shape

In [ ]:
wv_model.wv.most_similar('disaster')

In [ ]:
vocab=wv_model.wv.key_to_index
len(vocab)

In [ ]:
word_vec_dict={}
for word in vocab:
    word_vec_dict[word]=wv_model.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size
  

In [ ]:
# wv_model.wv.get_vector("like")

In [ ]:
reverse_vocab={v:k for k,v in vocab.items()}

In [ ]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(len(vocab)+1,100))
for word,i in vocab.items():
    embed_vector=wv_model.wv.get_vector(word)
    if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will st

In [ ]:
embed_matrix.shape

In [ ]:
# # from tensorflow.keras.preprocessing.text import Tokenizer
# # k_tokenizer = Tokenizer(oov_token='')

# # k_tokenizer.fit_on_texts(filtered_strings)
# # vocab=k_tokenizer.word_index



# tok = Tokenizer()
# tok.fit_on_texts(filtered_strings)
# vocab_size = len(vocab) + 1
# encd_rev = tok.texts_to_sequences(df['clean_review'])

In [ ]:
# vocab
final_sequences=[]
for i in filtered_strings:
    lis=tokenizer.tokenize(i)
    lis1=[]
    for i in lis:
        lis1.append(vocab[i])
    final_sequences.append(lis1)
final_sequences[0]

In [ ]:
vocab['forest']

In [ ]:
filtered_strings[0]

In [ ]:
reverse_vocab[124]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [ ]:
maxLen=max(len(x) for x in final_sequences)

In [ ]:
maxLen

In [ ]:
input_seq=np.array(pad_sequences(final_sequences,maxlen=maxLen,padding='pre'))

In [ ]:
# prepare train and val sets first
from sklearn.model_selection  import train_test_split
Y=keras.utils.to_categorical(df_train['target'])  # one hot target as required by NN.
x_train,x_val,y_train,y_val=train_test_split(input_seq,Y,test_size=0.20,random_state=42)

In [ ]:
total_words=len(vocab)+1

In [ ]:
total_words

In [ ]:
df_train.columns

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU,Flatten
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=total_words,output_dim=100,input_length=maxLen,embeddings_initializer=Constant(embed_matrix)))
# model.add(CuDNNLSTM(64,return_sequences=False)) # loss stucks at about 
model.add(LSTM(units=64,return_sequences=False))

model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.50))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(2,activation='sigmoid'))  # sigmod for bin. classification.

In [ ]:
model.summary()

In [ ]:
# compile the model
model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-3),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
# batch_size=64

In [ ]:
model.fit(x_train,y_train,epochs=epochs,validation_data=(x_val,y_val))


In [ ]:
# model=Sequential()
# model.add(Embedding(total_words,128,input_length=maxLen))
# # model.add(Bidirectional(LSTM(128)))
# # model.add(Dense(1,activation="Softmax"))
# model.add(LSTM(units=256,return_sequences=False))
# # model.add(LSTM(units=64,return_sequences=False))
# # model.add(Bidirectional(LSTM(64)))
# # model.add(LSTM(units=32))
# # Add a dense layer for sentiment analysis
# model.add(Dense(units=1, activation='sigmoid'))
# adam=Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])

# history=model.fit(input_seq,np.array(list(df_train['target'])),epochs=10)

# model